In [ ]:
import whisper

model = whisper.load_model("turbo")


In [ ]:
import sounddevice as sd
import numpy as np
import torch
import whisper
import torchaudio

def choose_microphone():
    try:
        devices = sd.query_devices()
        mic_list = [(i, device['name'], device['default_samplerate']) 
                    for i, device in enumerate(devices) if device['max_input_channels'] > 0]

        if not mic_list:
            print("No microphones found.")
            return None, None

        print("\nAvailable Microphones:")
        for i, (index, mic_name, rate) in enumerate(mic_list):
            print(f"{index}: {mic_name} (Default Sample Rate: {rate} Hz)")

        while True:
            try:
                mic_index = int(input("Enter the index of the microphone to use: "))
                for idx, _, rate in mic_list:
                    if mic_index == idx:
                        return mic_index, int(rate)
                print("Invalid selection. Please enter a valid index.")
            except ValueError:
                print("Invalid input. Please enter a number.")
    except Exception as e:
        print(f"An error occurred while listing microphones: {e}")
        return None, None

def record_and_transcribe(model, duration=5):
    mic_index, sample_rate = choose_microphone()
    if mic_index is None:
        return None, "No microphone selected."
    
    print(f"Using microphone index {mic_index} with sample rate {sample_rate} Hz for recording.")
    print(f"Recording for {duration} seconds...")

    try:
        # Set selected microphone and sample rate
        sd.default.device = mic_index
        audio_data = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=1, dtype='float32')
        sd.wait()
        print("Recording finished.")
    except Exception as e:
        print(f"Error while recording: {e}")
        return None, str(e)
    
    # Convert to tensor and normalize
    audio_tensor = torch.tensor(audio_data.flatten()).clamp(-1.0, 1.0)
    audio_tensor = audio_tensor.unsqueeze(0)

    # Save using torchaudio
    audio_path = "temp_audio.wav"
    torchaudio.save(audio_path, audio_tensor, sample_rate=sample_rate)

    # Transcribe with Whisper
    print("Transcribing audio...")
    result = model.transcribe(audio_path)
    
    return audio_tensor, result['text']

# Example Usage
model = whisper.load_model("base")


In [ ]:

audio_tensor, text = record_and_transcribe(model, duration=5)
if text:
    print("Transcribed Text:", text)

In [ ]:
import os
from dotenv import load_dotenv
from huggingface_hub import login

load_dotenv()  
HF_TOKEN = os.environ.get('HUGGING_FACE_HUB_TOKEN')

print(HF_TOKEN)

: 

In [ ]:
from generator import load_csm_1b

model = load_csm_1b()

In [ ]:
print(HF_TOKEN)